In [5]:
from pymongo import MongoClient


In [6]:
client = MongoClient('mongodb://localhost:27017/')
db = client['E-Commerce']
customers = db['customers']
products = db['products']
orders = db['orders']
orders_items = db['orders_items']

In [7]:
"""
Which product categories generate the highest revenue?
    Use $group to calculate revenue by category.
    Sort the results in descending order.
"""
pipeline1 = [
    {
        "$lookup": {
            'from': "order_items",
            'localField': 'order_id',
            'foreignField': 'order_id',
            'as': 'order_details'
            }
        },
        {
            '$unwind': '$order_details'
        },
        {
            '$lookup': {
                'from': 'products',
                'localField': 'order_details.product_id',
                'foreignField': 'product_id',
                'as': 'product_details'
            },
        },
        {
            '$unwind': '$product_details'
        },
        {
            '$group': {
                '_id': '$product_details.category',
                'total_revenue': {
                    '$sum': {
                        '$multiply': ['$order_details.quantity', '$order_details.price']
                    }
                }
            }
        }   
]
result = orders.aggregate(pipeline1)
for doc in result:
    print(doc)

{'_id': 'Accessories', 'total_revenue': 68956.55}
{'_id': 'Furniture', 'total_revenue': 35245.729999999996}
{'_id': 'Appliances', 'total_revenue': 12580.55}
{'_id': 'Electronics', 'total_revenue': 210480.24}
{'_id': 'Outdoors', 'total_revenue': 27474.0}


In [11]:
pipeline2 = [
    {
        "$lookup": {
            'from': "order_items",
            'localField': 'order_id',
            'foreignField': 'order_id',
            'as': 'order_details'
            }
        },
        {
            '$unwind': '$order_details'
        },
        {
            '$lookup': {
                'from': 'products',
                'localField': 'order_details.product_id',
                'foreignField': 'product_id',
                'as': 'product_details'
            },
        },
        {
            '$unwind': '$product_details'
        },
        {
            '$group': {
                '_id': '$product_details.category',
                'total_revenue': {
                    '$sum': {
                        '$multiply': ['$order_details.quantity', '$order_details.price']
                    }
                }
            }
        },
        {"$sort": {'total_revenue': -1}
        }
]
result = orders.aggregate(pipeline2)
for doc in result:
    print(doc)

{'_id': 'Electronics', 'total_revenue': 210480.24}
{'_id': 'Accessories', 'total_revenue': 68956.55}
{'_id': 'Furniture', 'total_revenue': 35245.729999999996}
{'_id': 'Outdoors', 'total_revenue': 27474.0}
{'_id': 'Appliances', 'total_revenue': 12580.55}


In [9]:
"""
Which states have the highest number of customers?
    Use $group to count customers by state.
    Sort the results in descending order.
"""
pipeline2 = [
    {
        '$group': {
            '_id': '$address.state',
            'total_customer': {'$sum': 1}
        }
    },
    {
        '$sort': {'total_customer': -1}
    }
]

result = customers.aggregate(pipeline2)
for doc in result:
    print(doc)

{'_id': 'Idaho', 'total_customer': 2}
{'_id': 'Ohio', 'total_customer': 2}
{'_id': 'Missouri', 'total_customer': 2}
{'_id': 'Maine', 'total_customer': 1}
{'_id': 'California', 'total_customer': 1}
{'_id': 'Mississippi', 'total_customer': 1}
{'_id': 'Wisconsin', 'total_customer': 1}
{'_id': 'New Mexico', 'total_customer': 1}
{'_id': 'Montana', 'total_customer': 1}
{'_id': 'Arizona', 'total_customer': 1}
{'_id': 'North Dakota', 'total_customer': 1}
{'_id': 'Florida', 'total_customer': 1}
{'_id': 'Massachusetts', 'total_customer': 1}
{'_id': 'Minnesota', 'total_customer': 1}
{'_id': 'West Virginia', 'total_customer': 1}
{'_id': 'Hawaii', 'total_customer': 1}
{'_id': 'Rhode Island', 'total_customer': 1}
{'_id': 'Tennessee', 'total_customer': 1}
{'_id': 'New Jersey', 'total_customer': 1}
{'_id': 'Wyoming', 'total_customer': 1}
{'_id': 'Illinois', 'total_customer': 1}


In [10]:
"""
What are the top 3 most expensive products sold in each order?
    Use $lookup and $sort to find the top products in each order.
"""
pipeline3 = [
    {
        '$group': {
            '_id': 'order_id',
            ''
        }
    }
    
]

SyntaxError: ':' expected after dictionary key (2370123394.py, line 9)

In [ ]:
pipeline = [
    {
        "$lookup": {
            "from": "order_items",  # Join with order_items collection
            "localField": "order_id",  # Use order_id from orders
            "foreignField": "order_id",  # Match with order_id from order_items
            "as": "order_details"  # Name the resulting array 'order_details'
        }
    },
    {
        "$unwind": "$order_details"  # Flatten the order_details array to deal with individual items
    },
    {
        "$lookup": {
            "from": "products",  # Join with the products collection
            "localField": "order_details.product_id",  # Match product_id from order_items
            "foreignField": "product_id",  # Match with product_id from products
            "as": "product_details"  # Name the resulting array 'product_details'
        }
    },
    {
        "$unwind": "$product_details"  # Flatten the product_details array
    },
    {
        "$sort": {
            "product_details.price": -1  # Sort the products in descending order by price
        }
    },
    {
        "$group": {
            "_id": "$order_id",  # Group by order_id
            "top_products": {  # Store the top products array
                "$push": {
                    "product_name": "$product_details.product_name",
                    "price": "$product_details.price"
                }
            }
        }
    },
    {
        "$project": {
            "top_products": {  # Only return the top 3 products
                "$slice": ["$top_products", 3]
            }
        }
    },
    {'$sort': {'_id':1}}
]

# Execute the aggregation pipeline
result = orders.aggregate(pipeline)

# Print the results
for doc in result:
    print(doc)


{'_id': 401, 'top_products': [{'product_name': 'Smartwatch', 'price': 250}, {'product_name': 'Keyboard', 'price': 50}]}
{'_id': 402, 'top_products': [{'product_name': 'Headphones', 'price': 200}, {'product_name': 'Earbuds', 'price': 100}]}
{'_id': 403, 'top_products': [{'product_name': 'TV', 'price': 1500}]}
{'_id': 404, 'top_products': [{'product_name': 'Refrigerator', 'price': 1000}, {'product_name': 'Microwave', 'price': 150}]}
{'_id': 405, 'top_products': [{'product_name': 'TV', 'price': 1500}]}
{'_id': 406, 'top_products': [{'product_name': 'Headphones', 'price': 200}]}
{'_id': 407, 'top_products': [{'product_name': 'Blender', 'price': 70}]}
{'_id': 408, 'top_products': [{'product_name': 'Tablet', 'price': 600}, {'product_name': 'Printer', 'price': 400}]}
{'_id': 409, 'top_products': [{'product_name': 'Gaming Console', 'price': 400}]}
{'_id': 410, 'top_products': [{'product_name': 'Smartphone', 'price': 800}]}
{'_id': 412, 'top_products': [{'product_name': 'Smartphone', 'price': 8